# scripts/attack.py

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
from argparse import Namespace
from datetime import datetime

import pandas as pd
import torch
from textattack import AttackArgs, Attacker
from textattack.attack_results import FailedAttackResult, SkippedAttackResult
from textattack.datasets import Dataset

import nlp_adversarial_attacks.attack.utils as utils
from nlp_adversarial_attacks.attack import (
    ModelWrapper,
    get_attack_recipe,
    predict,
    save_results,
)
from nlp_adversarial_attacks.models.model_loading import load_target_model

In [3]:
args = {
    # I/O settings
    "dir_dataset": "data/",
    "dir_out": "attacks/",
    # Experiment settings
    "task_name": "sentiment",
    # Model parameters
    "model_name": "distilcamembert",
    "pretrained_model_name_or_path": "baptiste-pasquier/distilcamembert-allocine",
    "model_max_seq_len": 512,
    "model_batch_size": 32,
    # Data parameters
    "dataset_name": "allocine",
    "target_model_train_dataset": "allocine",
    # Attack parameters
    "attack_toolchain": "textattack",
    "attack_name": "deepwordbug",
    "attack_query_budget": 0,
    "attack_n_samples": 10,
    # Other parameters
    "random_seed": 0,
}

args = Namespace(**args)

In [4]:
start = time.time()

In [5]:
# setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(args.random_seed)

In [6]:
# setup output directory
out_dir = os.path.join(
    args.dir_out,
    args.dataset_name,
    args.model_name,
    args.attack_toolchain,
    args.attack_name,
    str(args.attack_n_samples),
)
os.makedirs(out_dir, exist_ok=True)
logger = utils.get_logger(os.path.join(out_dir, "log.txt"))
logger.info(args)
logger.info(f"Timestamp: {datetime.now()}")

Namespace(dir_dataset='data/', dir_out='attacks/', task_name='sentiment', model_name='distilcamembert', pretrained_model_name_or_path='baptiste-pasquier/distilcamembert-allocine', model_max_seq_len=512, model_batch_size=32, dataset_name='allocine', target_model_train_dataset='allocine', attack_toolchain='textattack', attack_name='deepwordbug', attack_query_budget=0, attack_n_samples=10, random_seed=0)
Timestamp: 2023-03-12 16:41:35.465951


In [7]:
# save config file
utils.cmd_args_to_yaml(args, os.path.join(out_dir, "config.yml"))

In [8]:
# set no. labels
num_labels = 2
if args.dataset_name in ["nuclear_energy", "climate-change_waterloo"]:
    num_labels = 3

In [9]:
# load trained model
logger.info("\nLoading trained model...")
model = load_target_model(
    model_name=args.model_name,
    pretrained_model_name_or_path=args.pretrained_model_name_or_path,
    num_labels=num_labels,
    max_seq_len=args.model_max_seq_len,
    device=device,
)
model


Loading trained model...


AutoClassifier(
  (classifier): CamembertForSequenceClassification(
    (roberta): CamembertModel(
      (embeddings): CamembertEmbeddings(
        (word_embeddings): Embedding(32005, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): CamembertEncoder(
        (layer): ModuleList(
          (0): CamembertLayer(
            (attention): CamembertAttention(
              (self): CamembertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): CamembertSelfOutput(
           

In [10]:
# Example
text_list = [
    "Mon dieu !!!!!!! Ce film à sacrément vieilli",
    "Super film",
]

with torch.no_grad():
    logits = model(text_list)
logits

tensor([[ 2.8276, -2.5389],
        [-3.4611,  3.0037]], device='cuda:0')

In [11]:
# read in the test set
logger.info("\nLoading test data set...")
dir_dataset = os.path.join(args.dir_dataset, args.dataset_name)

if os.path.exists(os.path.join(dir_dataset, "test.csv")):
    test_df = pd.read_csv(os.path.join(dir_dataset, "test.csv"))

elif os.path.join(dir_dataset, "data.csv"):
    test_df = pd.read_csv(os.path.join(dir_dataset, "data.csv"))

if "index" in test_df.columns:
    del test_df["index"]

assert test_df.columns.tolist() == ["text", "label"]
test_df = test_df.reset_index().rename(columns={"index": "test_index"})

test_df


Loading test data set...


,test_index,text,label
0,0,"Magnifique épopée, une belle histoire, touchan...",1
1,1,Je n'ai pas aimé mais pourtant je lui mets 2 é...,0
2,2,Un dessin animé qui brille par sa féerie et se...,1
3,3,"Si c'est là le renouveau du cinéma français, c...",0
4,4,Et pourtant on s’en Doutait !Second volet très...,0
...,...,...,...
19995,19995,"je suis éventreur, arracheur, tailladeur, goug...",1
19996,19996,Trémors 3 essouffle la série des trémors par u...,0
19997,19997,"0/20 : Tout d’abord, la mise en scène est tout...",0
19998,19998,Un scénario très original mené par des personn...,1


In [12]:
# generate predictions on the test set
t = time.time()
logger.info("Making prediction on the test set...")
pred_test, proba_test = predict(args, model, test_df, device, logger=logger)
test_df["pred"] = pred_test
logger.info(f"Time: {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")

test_df

Making prediction on the test set...
No. test samples: 20,000


  0%|          | 0/625 [00:00<?, ?it/s]

Accuracy: 0.971
Time: 00:01:06


,test_index,text,label,pred
0,0,"Magnifique épopée, une belle histoire, touchan...",1,1
1,1,Je n'ai pas aimé mais pourtant je lui mets 2 é...,0,0
2,2,Un dessin animé qui brille par sa féerie et se...,1,1
3,3,"Si c'est là le renouveau du cinéma français, c...",0,0
4,4,Et pourtant on s’en Doutait !Second volet très...,0,0
...,...,...,...,...
19995,19995,"je suis éventreur, arracheur, tailladeur, goug...",1,1
19996,19996,Trémors 3 essouffle la série des trémors par u...,0,0
19997,19997,"0/20 : Tout d’abord, la mise en scène est tout...",0,0
19998,19998,Un scénario très original mené par des personn...,1,1


In [13]:
# get label names and encode labels if necessary
if args.dataset_name in [
    "wikipedia",
    "civil_comments",
    "hatebase",
    "wikipedia_personal",
    "wikipedia_aggression",
    "reddit_dataset",
    "gab_dataset",
]:
    label_names = ["non-toxic", "toxic"]

elif args.dataset_name == "fnc1":
    label_map = {"agree": 0, "disagree": 1, "discuss": 2, "unrelated": 3}
    inverse_label_map = {v: k for k, v in label_map.items()}
    label_names = [inverse_label_map[i] for i in range(len(inverse_label_map))]

elif args.dataset_name in ["imdb", "sst", "allocine"]:
    label_names = ["negative", "positive"]

elif args.dataset_name in ["nuclear_energy", "climate-change_waterloo"]:
    label_names = ["negative", "neutral", "positive"]

else:
    raise ValueError(f"unknown dataset {args.dataset_name}")

In [14]:
# focus on correctly predicted TOXIC instances for abuse
if args.task_name == "abuse":
    temp_df = test_df[(test_df["label"] == 1) & (test_df["pred"] == 1)].copy()

# focus on correctly predicted instances
else:
    temp_df = test_df[test_df["label"] == test_df["pred"]].copy()
temp_df

,test_index,text,label,pred
0,0,"Magnifique épopée, une belle histoire, touchan...",1,1
1,1,Je n'ai pas aimé mais pourtant je lui mets 2 é...,0,0
2,2,Un dessin animé qui brille par sa féerie et se...,1,1
3,3,"Si c'est là le renouveau du cinéma français, c...",0,0
4,4,Et pourtant on s’en Doutait !Second volet très...,0,0
...,...,...,...,...
19995,19995,"je suis éventreur, arracheur, tailladeur, goug...",1,1
19996,19996,Trémors 3 essouffle la série des trémors par u...,0,0
19997,19997,"0/20 : Tout d’abord, la mise en scène est tout...",0,0
19998,19998,Un scénario très original mené par des personn...,1,1


In [15]:
# attack text prioritizing longer text instances
# temp_df["length"] = temp_df["text"].apply(lambda x: len(x.split()))
# temp_df = temp_df.sort_values("length", ascending=False)
if args.attack_n_samples:
    temp_df = temp_df[: args.attack_n_samples]
temp_df = temp_df.reset_index(drop=True)
logger.info("\nBeginning attacks...")
logger.info(f"No. test: {len(test_df):,}, no. candidates: {len(temp_df):,}")


Beginning attacks...
No. test: 20,000, no. candidates: 10


In [16]:
# result containers
results = []
t = time.time()  # total time

# selected indices information
y_test = temp_df["label"].values
test_indices = temp_df["test_index"].values

In [17]:
# TextAttack

# prepare data
dataset = Dataset(list(zip(temp_df["text"], temp_df["label"])), label_names=label_names)

# prepare attacker
model_wrapper = ModelWrapper(model)
attack_recipe = get_attack_recipe(args.attack_toolchain, args.attack_name)
attack = attack_recipe.build(model_wrapper)
attack_args = AttackArgs(
    num_examples=-1,
    query_budget=args.attack_query_budget,
    log_summary_to_json=os.path.join(out_dir, "summary.json"),
)
attacker = Attacker(attack, dataset, attack_args)

textattack: Unknown if model of class <class 'transformers.models.camembert.modeling_camembert.CamembertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [18]:
attack_results = attacker.attack_dataset()

textattack: Logging Summary to JSON at path attacks/allocine\distilcamembert\textattack\deepwordbug\10\summary.json


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [00:00<00:05,  1.80it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (86%)]]

[[Magnifique]] [[épopée]], [[une]] belle [[histoire]], [[touchante]] avec des [[acteurs]] qui interprètent très bien leur rôles (Mel Gibson, Heath Ledger, Jason Isaacs...), le genre de [[film]] [[qui]] se [[savoure]] en [[famille]]! :)

[[Mcgnifique]] [[éApopée]], [[ne]] belle [[ihstoire]], [[tuochante]] avec des [[acteur]] qui interprètent très bien leur rôles (Mel Gibson, Heath Ledger, Jason Isaacs...), le genre de [[filMm]] [[Wui]] se [[savorue]] en [[afmille]]! :)




[Succeeded / Failed / Skipped / Total] 3 / 0 / 0 / 3:  30%|███       | 3/10 [00:00<00:02,  3.09it/s]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (55%)]]

Un [[très]] [[joli]] [[film]], qui [[ressemble]] à un téléfilm mais [[qui]] a le mérite d'être [[émouvant]] et [[proche]] de ses personnages. Magimel est [[vraiment]] [[très]] bon [[et]] [[l'histoire]] est [[touchante]]

Un [[Vrès]] [[Roli]] [[flm]], qui [[ressempble]] à un téléfilm mais [[Equi]] a le mérite d'être [[méouvant]] et [[pMroche]] de ses personnages. Magimel est [[vraimAnt]] [[rès]] bon [[te]] [[l'ihstoire]] est [[toucante]]




[Succeeded / Failed / Skipped / Total] 7 / 2 / 0 / 9:  90%|█████████ | 9/10 [00:19<00:02,  2.21s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (100%)]] --> [[[FAILED]]]

Mais comment certaines personnes ont pus lui mettre 5/5 et donc dire indirectement que c'est un chef-d'œuvre ??? Et comment a-t-il fait pour sortir au cinéma et non en DTV ??? C'est pas un film que l'on regarde dans une salle obscur ça, pour moi ça ressemble plus à un téléfilm que l'on visionne un dimanche pluvieux pour que les enfants arrête de nous casser les pieds ! Et puis, le scénario avec le chien que devient le meilleur ami du gosse, c'est du vu et revu (un cliché) ! L'acteur principal est quant à lui aussi agaçant que son personnage ! Les suites ont l'air aussi mauvaises que Buddy Star des Paniers étant donné que l'histoire est quasiment la même (pour moi ça c'est pas des suites, c'est plutôt une succession de petits reboots inutiles). Reste regardable pour les moins de 10 ans (et encore, même moi à 6 ans, je n'aurais pas aimé).




[Succeeded / Failed / Skipped / Total] 8 / 2 / 0 / 10: 100%|██████████| 10/10 [00:20<00:00,  2.06s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (61%)]]

[[LE]] [[film]] de mon enfance , il a un peu vieilli maintenant , mais l'ours reste toujours impressionnant, il est bien réel contrairement au film 'the Revenant" . Ce n'est surement pas un [[chef-d'œuvre]] [[mais]] je le trouve bien réalise , captivant , beaux et accompagné d'une superbe musique. Le gros points noir c'est la facilité qu'ils ont a créer des peaux , des pièges , et rester longtemps sans manger....mais on oublie assez vite ces [[erreurs]] grâce a un casting sympathique et aux décors naturels. Un vieux [[film]] mais qui reste [[toujours]] [[un]] [[bon]] [[film]].

[[dE]] [[dilm]] de mon enfance , il a un peu vieilli maintenant , mais l'ours reste toujours impressionnant, il est bien réel contrairement au film 'the Revenant" . Ce n'est surement pas un [[che-fd'œuvre]] [[masi]] je le trouve bien réalise , captivant , beaux et acco

In [19]:
# attack test set
for i, attack_result in enumerate(attack_results):
    # get attack status
    status = "success"
    if isinstance(attack_result, FailedAttackResult):
        status = "failed"
    elif isinstance(attack_result, SkippedAttackResult):
        status = "skipped"

    # get original and peturbed objects
    og = attack_result.original_result
    pp = attack_result.perturbed_result

    num_words_changed = len(og.attacked_text.all_words_diff(pp.attacked_text))

    result = {
        "target_model_train_dataset": args.target_model_train_dataset,
        "target_dataset": args.dataset_name,
        "target_model": args.model_name,
        "attack_name": args.attack_name,
        "test_index": test_indices[i],
        "ground_truth": y_test[i],
        "status": status,
        "original_text": og.attacked_text.text,
        "original_output": og.raw_output.tolist(),
        "perturbed_text": pp.attacked_text.text,
        "perturbed_output": pp.raw_output.tolist(),
        "num_queries": attack_result.num_queries,
    }
    try:
        result["frac_words_changed"] = num_words_changed / len(og.attacked_text.words)
    except ZeroDivisionError:
        result["frac_words_changed"] = -1
    results.append(result)

    s = "Result {} (dataset={}, model={}, attack={}, no. queries={:,}):"
    logger.info(
        s.format(
            i + 1,
            args.dataset_name,
            args.model_name,
            args.attack_name,
            result["num_queries"],
        )
    )
    logger.info(attack_result.goal_function_result_str() + "\n")
    # logger.info(attack_result.__str__(color_method="ansi"))

# save leftover results
logger.info(f"\nSaving results to {out_dir}/...")
save_results(args, results, out_dir, logger, done=True)

Result 1 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=82):
Positive (100%) --> Negative (86%)

Result 2 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=65):
Negative (62%) --> Positive (99%)

Result 3 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=24):
Positive (99%) --> Negative (98%)

Result 4 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=57):
Negative (100%) --> Positive (71%)

Result 5 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=76):
Negative (100%) --> Positive (57%)

Result 6 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=504):
Positive (100%) --> [FAILED]

Result 7 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=89):
Positive (100%) --> Negative (88%)

Result 8 (dataset=allocine, model=distilcamembert, attack=deepwordbug, no. queries=93):
Positive (100%) --> Negative (55%)

Result 9 (datas

In [20]:
logger.info(f"\nAttack time: {time.strftime('%H:%M:%S', time.gmtime(time.time()-t))}")
logger.info(f"Total time: {time.strftime('%H:%M:%S', time.gmtime(time.time()-start))}")


Attack time: 00:00:21
Total time: 00:01:32


In [21]:
pd.read_csv("attacks/allocine/distilcamembert/textattack/deepwordbug/10/results.csv")

,scenario,target_dataset,target_model,attack_toolchain,attack_name,original_text,perturbed_text,original_output,perturbed_output,status,target_model_train_dataset,test_index,ground_truth,num_queries,frac_words_changed,attacked_all_instances
0,sentiment,allocine,distilcamembert,textattack,deepwordbug,"Magnifique épopée, une belle histoire, touchan...","Mcgnifique éApopée, ne belle ihstoire, tuochan...","[0.00047096062917262316, 0.9995290040969849]","[0.8554291725158691, 0.14457082748413086]",success,allocine,0,1,82,0.333333,True
1,sentiment,allocine,distilcamembert,textattack,deepwordbug,Je n'ai pas aimé mais pourtant je lui mets 2 é...,Je n'ai pas imé mais pourtant je lui mets 2 ét...,"[0.6205608248710632, 0.3794391453266144]","[0.01314047072082758, 0.9868595600128174]",success,allocine,1,0,65,0.016393,True
2,sentiment,allocine,distilcamembert,textattack,deepwordbug,Un dessin animé qui brille par sa féerie et se...,nU dessin animé qui brillBe par sa féerim et s...,"[0.006190015003085136, 0.9938099980354309]","[0.981940746307373, 0.018059266731142998]",success,allocine,2,1,24,0.272727,True
3,sentiment,allocine,distilcamembert,textattack,deepwordbug,"Si c'est là le renouveau du cinéma français, c...","Ti Ec'est là le renouveau du cinéma français, ...","[0.999725878238678, 0.00027412595227360725]","[0.29408329725265503, 0.705916702747345]",success,allocine,3,0,57,0.250000,True
4,sentiment,allocine,distilcamembert,textattack,deepwordbug,Et pourtant on s’en Doutait !Second volet très...,t plurtant on s’eU Doutait !Sceond olet très m...,"[0.9996825456619263, 0.00031743431463837624]","[0.43405482172966003, 0.5659451484680176]",success,allocine,4,0,76,0.588235,True
5,sentiment,allocine,distilcamembert,textattack,deepwordbug,Vous reprendrez bien un peu d'été ? Ce film je...,Vous reprendrez bien un peu d'été ? Ce film je...,"[0.0004930670256726444, 0.999506950378418]","[0.37118270993232727, 0.6288173198699951]",failed,allocine,5,1,504,0.072072,True
6,sentiment,allocine,distilcamembert,textattack,deepwordbug,"Terrible histoire que ces êtres sans amour, ce...","Terirble histoire que ces êtres sans amour, ce...","[0.0014063301496207714, 0.9985936284065247]","[0.88441002368927, 0.11558996886014938]",success,allocine,7,1,89,0.041096,True
7,sentiment,allocine,distilcamembert,textattack,deepwordbug,"Un très joli film, qui ressemble à un téléfilm...","Un Vrès Roli flm, qui ressempble à un téléfilm...","[0.0022956894244998693, 0.9977043271064758]","[0.5488104820251465, 0.45118948817253113]",success,allocine,8,1,93,0.400000,True
8,sentiment,allocine,distilcamembert,textattack,deepwordbug,Mais comment certaines personnes ont pus lui m...,Mais commewt certaines personnes ont pus liu m...,"[0.9997686743736267, 0.0002313780423719436]","[0.9495946168899536, 0.05040530115365982]",failed,allocine,9,0,301,0.165563,True
9,sentiment,allocine,distilcamembert,textattack,deepwordbug,"LE film de mon enfance , il a un peu vieilli m...","dE dilm de mon enfance , il a un peu vieilli m...","[0.000899123027920723, 0.999100923538208]","[0.6063792705535889, 0.39362069964408875]",success,allocine,10,1,145,0.111111,True
